# Project 2

This report is an overview of the work done in project 2. It starts with a highlight of some sections of the command line history (to show what the commands are doing) then continues onto an explanation of the pyspark code and finally concludes with three simple business questions being answered with queries.

### Publishing Data with Kafka

The first steps in this project are to publish the data into kafka so it can be consumed by others. Therefore, the first step is to set up the kafka queue and publish messages to it. However, before we do that, we need to start up our cluster so we can use kafka. This is done through the command line by the following code (navigating to the correct directory and starting up our cluster)
```
cd w205/project-2-isaacvernon1/  
docker-compose up -d
```
This will start up the cluster specified in our docker-compose.yml file, which includes zookeeper (which kafka depends on), kafka, cloudera (for Hadoop), spark, and a mids image. Now that we have intialized our cluster, we can check to make sure everything is running properly with
```
docker-compose ps 
docker ps -a
```
We could also use the following code in another linux command line window to watch the logs from Kafka in real time as they come in (we need to make sure we are in the w205/project-2-isaacvernon1/ directory)
```
docker-compose logs -f kafka
```
Now that we have our Kafka cluster started, we need to create a topic. I create a topic assessments, with the name coming from the fact that the data we are going to publish is assessment data (We can use the second line of code to determine if our topic was created successfully). 
```
docker-compose exec kafka kafka-topics --create --topic assessments --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181

docker-compose exec kafka kafka-topics --describe --topic assessments --zookeeper zookeeper:32181
```
Now that we have created a kafka topic, it is time to publish some messages to the queue. We do so using our assessments data and kafkacat to write to the queue (second line of code will print the messages that were published to kafka to the command line).
```
docker-compose exec mids bash -c "cat /w205/project-2-isaacvernon1/assessment-attempts-20180128-121051-nested.json | jq '.[]' -c | kafkacat -P -b kafka:29092 -t assessments"

docker-compose exec mids bash -c "kafkacat -C -b kafka:29092 -t assessments -o beginning -e"
```
Finally, we now have to set up our system so we can write our pyspark code in a Jupyter Notebook (much more convenient as compared to writing in console). We run the following code and eventually get a link where we replace the 0.0.0.0 with our external IP address from our Virtual Machine.
```
docker-compose exec spark bash
# These two lines are in the spark shell, exit will quit for us
ln -s /w205 w205
exit

docker-compose exec spark env PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS='notebook --no-browser --port 8888 --ip 0.0.0.0 --allow-root' pyspark
```
We are now set to run the rest of our pyspark commands in a Jupyter Notebook, which is SparkCode.ipynb.

### Consuming Data with Pyspark

Now that we have our pyspark session set up in a Jupyter Notebook, we can consume the messages from Kafka. This is done through the following code. We can also cache this data to help prevent future warning messages as well as casting the messages as strings into a new dataframe to prepare for future code.
```
raw_assessments = spark.read.format("kafka").option("kafka.bootstrap.servers", "kafka:29092").option("subscribe","assessments").option("startingOffsets", "earliest").option("endingOffsets", "latest").load()

raw_assessments.cache()

assessments = raw_assessments.select(raw_assessments.value.cast('string'))
```

### Transforming the Data and Creating Tables with Pyspark

Now that we have loaded the data into pyspark, we can now transform our intial data to create queryable dataframes. In this part, we will create 3 different data tables; assessments, scores, and questions. For the first data table, we want to create a table that contains some of the external Json layer information. This only requires the simplest transformation. With the following line of code, we can unroll the data to expose the outermost values (will still have nesting in sequences, so we can't use these values in the table). We can then register and name our data table as a temporary table "assessments" which we can later run SQL queries against.
```
unroll_assessments = assessments.rdd.map(lambda x: Row(**json.loads(x.value))).toDF()

unroll_assessments.registerTempTable('assessments')
```
Next, we will create the scores table. When we look at the structure of the nested Json and what fields we may be interested to act upon, we can see that in the nested sequence there are counts of questions, and the number of correct questions. We will take these two fields (and combine them to make a percent column that represents the percentage of correct answers for the test) and the name of the exam for analysis. The below code will use the function lambda_scores (for the construction of this function see SparkCode.ipynb) to unroll the Json data in this manner and save the result as a temporary table "scores".
```
my_scores = assessments.rdd.flatMap(lambda_scores).toDF()

my_scores.registerTempTable('scores')
```
Finally, we will also create a questions table. Within the nesting in sequences there is a nested list called questions. Within the data for each question is a unique question identifier. Using this question id, along with a sequence id (pertains to the sequence of questions), user id, exam name, and question number we can create a new table questions. We will use the function lambda_questions to do so (built in SparkCode.ipynb) and will save the result as a temporary table "questions".
```
my_questions = assessments.rdd.flatMap(lambda_questions).toDF()

my_questions.registerTempTable('questions')
```

### Running SQL Queries and Answering Business Questions with Pyspark

Now that we have temporary tables saved we can go ahead and run SQL queries against them. This will let us answer some simple business questions and think about what we can use the tables for. There are three business questions that will be answered in this part:

1) How many people took each exam?

2) What is the average and standard deviation of scores on all exams?

3) For a particular user_exam_id (use '6d4089e4-bde5-4a22-b65f-18bce9ab79c8' as an example), what are the id's of the questions asked? Sequence Id?

The first question can be answered using the "assessments" table. When we look at the assessments table, we can see a pretty glaring issue. The sequence column in this table has nested values, which means that we won't be able to use this (or anything inside) for querying from this table. However, when we look at our first business question, we can see that we won't need to use this. To answer the question of how many people took each exam, we use the spark sql statement below.
```
spark.sql("select count(*), exam_name from assessments group by exam_name").show()
```
This will give us a table of two columns: exam_name and count. Now, if we want to know how many people took an assessment, we can just look at this output, or we could even make the sql even more specific to get the numbers for a specific exam. We find ranges for our table from 1 person taking the assessment to 394 people taking "Learning Git".

The second question asks for the average and standard deviation of scores on all exams. We can use our created table "scores" to answer this question. In the making of "scores", we throw away any observations that don't have data on the total number of questions or the number of correct questions since artificially inserting 0's would change our totals for these values. Using the below code, we can run a spark sql queries to answer our question.
```
spark.sql("select avg(percent) from scores").show()
spark.sql("select stddev(percent) from scores").show()
```
We find that the average over all exams with data is 62.66 percent with a standard deviation of 31.09 percent. This could be informative if we are looking to see how difficult the exams are on average. We could also modify the queries (groupby exam_name) to see the scores on each test.

Finally, the third question asks for the id's of the questions for a given user_exam_id. This could help us to investigate a particular individual to get some sort of information about their circumstances. This can then be performed by the following code (with the user_exam_id being a an example).
```
spark.sql("select * from questions where user_exam_id == '6d4089e4-bde5-4a22-b65f-18bce9ab79c8'").show()
```
We find that this user took an exam with four questions, each with their own id. If we want to investigate further, we can try to see what those questions asked exactly with some outside data (from further data sources).

### Writing to HDFS with Pyspark

Now that we have queried our dataframes (and made sure that they answer our questions), we can write our dataframes to Hadoop in parquet format. The following code will write each of the three tables to HDFS as "assessments", "scores", and "questions" through cloudera (see the docker-compose.yml).
```
unroll_assessments.write.parquet("/tmp/assessments")

my_scores.write.parquet("/tmp/scores")

my_questions.write.parquet("/tmp/questions")
```
We can confirm that the table was correctly written to HDFS with the following command on a second command line window (make sure we are in w205/project-2-isaacvernon1/ directory).
```
docker-compose exec cloudera hadoop fs -ls /tmp/
```

### Tearing Down the Cluster

Since we have now finished with our work in pyspark, we can now exit and tear down the cluster. We first exit the pyspark instance (ctrl c followed by y) and can now bring down our cluster with the following code in the command line.
```
docker-compose down
```
We can check to make sure our cluster has been brought down with
```
docker-compose ps
docker ps -a
```
Now that we have ensured our cluster is down and there are no stray containers, the project is complete